In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from collections import defaultdict
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
from transformers import pipeline
import itertools
import torch

In [ ]:
df = pd.read_csv("../data/sample/sample_processed_reviews.csv")
df

,reviews,sentiment,processed_reviews
0,Огромная благодарность Анне Геннадьевне Рябцев...,1,огромный благодарность анне геннадиевич рябцев...
1,"Казахстан, г. Алматы, тёща ходила на Серагем 2...",0,казахстан г алматы тща ходить серагема 2 3 год...
2,"Пусть будет немного странно, но хотела бы выра...",1,пусть немного странный хотеть выразить благода...
3,Мне очень понравилось отношение к пациенту. Пр...,1,очень понравиться отношение пациент предельный...
4,"Ну, лично я считаю, что для бесплатной консуль...",1,лично считать бесплатный консультация вполне с...
...,...,...,...
9995,Были 02 февраля в клинике впервые. Очень понра...,1,02 февраль клиника впервые очень понравиться и...
9996,"В течении 2 лет обращалась с температурой, сда...",0,течение 2 год обращаться температура сдавать а...
9997,"Анна Аштовотовна не просто специалист от бога,...",1,анна аштовотович просто специалист бог врач зо...
9998,В течение 2 месяцев не могли установить онколо...,0,течение 2 месяц мочь установить онкология 68 л...


In [41]:
docs = df['processed_reviews'].tolist()

In [4]:
model = SentenceTransformer('all-MiniLM-L12-v2', device='mps')
embeddings = model.encode(docs, batch_size=256, show_progress_bar = True)

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

In [42]:
np.save("healthcare_facilities_reviews_embeddings.npy", embeddings)

## 2. Model Training

In [ ]:
embeddings = np.load("../data/healthcare_facilities_reviews_embeddings.npy")

### 2.1.1 BERTopic

In [44]:
umap_model = UMAP(
    n_neighbors = 15,
    n_components = 5,
    min_dist = 0.0,
    metric = 'cosine',
    low_memory = True
)           

hdbscan_model = HDBSCAN(
    min_cluster_size=30,
    metric="euclidean",
    cluster_selection_method="eom"
)

In [45]:
bertopic_model = BERTopic(
    language="multilingual", 
    umap_model = umap_model,
    hdbscan_model = hdbscan_model,
    calculate_probabilities=False)

In [46]:
topics, probs = bertopic_model.fit_transform(docs, embeddings)

In [47]:
bertopic_model.get_topic_info().head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,661,-1_врач_весь_это_день,"[врач, весь, это, день, сказать, ребенок, отде...",[16082010 утро жк дать направление 17 роддом т...
1,0,5705,0_врач_весь_очень_это,"[врач, весь, очень, это, клиника, спасибо, кот...",[клиника несколько работать настоящий врач лор...
2,1,763,1_врач_весь_это_очень,"[врач, весь, это, очень, клиника, жизнь, сказа...",[хотеть рассказать мнение клиника жена ходить ...
3,2,701,2_год_весь_врач_это,"[год, весь, врач, это, отделение, ребенок, ден...",[02 октябрь 2012 родить второй роддом долго вы...
4,3,381,3_весь_врач_это_сказать,"[весь, врач, это, сказать, отделение, день, ре...",[начаться рождение малыш родиться новогодний п...


In [48]:
original_reviews = df["reviews"]
bert_topic_to_reviews = defaultdict(list)

for review, topic_id in zip(original_reviews, topics):
    bert_topic_to_reviews[topic_id] = review

### 2.1.2 LDA

In [49]:
tokenised_reviews = [doc.split() for doc in docs]

dictionary = corpora.Dictionary(tokenised_reviews)

corpus = [dictionary.doc2bow(token) for token in tokenised_reviews]

In [53]:
num_topics_list = [15, 20, 25]
alpha_list = ['auto', 0.1, 0.5]
eta_list = ['auto', 0.1, 0.5]

best_model = None
best_coherence = -1

for num_topics, alpha, eta in itertools.product(num_topics_list, alpha_list, eta_list):
    lda = LdaModel(corpus=corpus, id2word=dictionary, 
                   num_topics=num_topics, alpha=alpha, eta=eta,
                   passes=10, random_state=42)
    coherence_model = CoherenceModel(model=lda, texts=tokenised_reviews, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    print(num_topics, alpha, eta, coherence)
    if coherence > best_coherence:
        best_coherence = coherence
        best_model = lda

15 auto auto 0.5817964287146807
15 auto 0.1 0.6071915891813656
15 auto 0.5 0.4472889903750628
15 0.1 auto 0.5548821777106281
15 0.1 0.1 0.5836439748020633
15 0.1 0.5 0.41348413878542495
15 0.5 auto 0.43803382406859337
15 0.5 0.1 0.5364469815489162
15 0.5 0.5 0.4115416814989305
20 auto auto 0.5524783112841479
20 auto 0.1 0.6245479925888927
20 auto 0.5 0.4362463992209976
20 0.1 auto 0.5310420353733213
20 0.1 0.1 0.6130880671535606
20 0.1 0.5 0.4161509829454147
20 0.5 auto 0.4336442403029899
20 0.5 0.1 0.5792614826602439
20 0.5 0.5 0.4081023904743737
25 auto auto 0.5743170953519126
25 auto 0.1 0.6320281777237766
25 auto 0.5 0.4468980398027822
25 0.1 auto 0.5594378551952452
25 0.1 0.1 0.6105779544744095
25 0.1 0.5 0.39425274955452994
25 0.5 auto 0.39503456112659835
25 0.5 0.1 0.5698478890446387
25 0.5 0.5 0.5099118134746553


In [83]:
num_topics = 20

lda_model = LdaModel(
    corpus = corpus,
    id2word = dictionary,
    num_topics = num_topics,
    random_state = 42,
    passes = 10,
    alpha = 0.5,
    per_word_topics = True,
    eta = 0.1
)

In [84]:
lda_topics = []

for bow in corpus:
    topic_probs = lda_model.get_document_topics(bow)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    lda_topics.append(dominant_topic)

In [85]:
lda_topic_to_reviews = defaultdict(list)

for review, topic_id in zip(original_reviews, lda_topics):
    lda_topic_to_reviews[topic_id].append(review)

for topic, reviews in lda_topic_to_reviews.items():
    print(f"Topic {topic}: {len(lda_topic_to_reviews[topic])} docs")

Topic 3: 2507 docs
Topic 13: 2902 docs
Topic 1: 1621 docs
Topic 4: 2161 docs
Topic 7: 208 docs
Topic 19: 256 docs
Topic 18: 188 docs
Topic 15: 119 docs
Topic 17: 28 docs
Topic 6: 7 docs
Topic 12: 3 docs


### 2.2 Model Evaluation

In [86]:
coherence_model = CoherenceModel(
    model=lda_model, 
    texts=tokenised_reviews, 
    dictionary=dictionary, 
    coherence='c_v')

coherence_score = coherence_model.get_coherence()
print("LDA Coherence Score:", coherence_score)

LDA Coherence Score: 0.5792614826602439


In [18]:
topic_words = [[w for w,_ in bertopic_model.get_topic(t)] 
                for t in bertopic_model.get_topic_info().Topic if t != -1]

coherence_model = CoherenceModel(
    topics=topic_words,
    texts=tokenised_reviews,
    dictionary=dictionary, 
    coherence='c_v'
)

coherence_score = coherence_model.get_coherence()
print("BERTopic Coherence Score:", coherence_score)

BERTopic Coherence Score: 0.3775813239971198


## 2.3 Model Selection & Topic ID Assignment

In [87]:
df['topic_id'] = lda_topics

In [88]:
df.head()

,reviews,sentiment,processed_reviews,topic_id
0,Огромная благодарность Анне Геннадьевне Рябцев...,1,огромный благодарность анне геннадиевич рябцев...,3
1,"Казахстан, г. Алматы, тёща ходила на Серагем 2...",0,казахстан г алматы тща ходить серагема 2 3 год...,13
2,"Пусть будет немного странно, но хотела бы выра...",1,пусть немного странный хотеть выразить благода...,3
3,Мне очень понравилось отношение к пациенту. Пр...,1,очень понравиться отношение пациент предельный...,3
4,"Ну, лично я считаю, что для бесплатной консуль...",1,лично считать бесплатный консультация вполне с...,13


## 3 Topic Summarisation

### 3.1 Representatives Selection

In [89]:
lda_topics_probs = []

for bow in corpus:
    topic_probs = lda_model.get_document_topics(bow)
    dominant_topic_prob = max(topic_probs, key=lambda x: x[1])[1]
    lda_topics_probs.append(dominant_topic_prob)

In [90]:
df["topic_confidence"] = lda_topics_probs

In [91]:
df.head(10)

,reviews,sentiment,processed_reviews,topic_id,topic_confidence
0,Огромная благодарность Анне Геннадьевне Рябцев...,1,огромный благодарность анне геннадиевич рябцев...,3,0.452537
1,"Казахстан, г. Алматы, тёща ходила на Серагем 2...",0,казахстан г алматы тща ходить серагема 2 3 год...,13,0.337037
2,"Пусть будет немного странно, но хотела бы выра...",1,пусть немного странный хотеть выразить благода...,3,0.468332
3,Мне очень понравилось отношение к пациенту. Пр...,1,очень понравиться отношение пациент предельный...,3,0.249642
4,"Ну, лично я считаю, что для бесплатной консуль...",1,лично считать бесплатный консультация вполне с...,13,0.394573
5,Первый раз попали к педиатру Чеглаковой Евгени...,1,первый попасть педиатр чеглаков евгения валери...,3,0.303528
6,"Сейчас столько возможностей похудеть, давно эт...",1,столько возможность похудеть давно это переста...,13,0.301174
7,"Отношение неплохое, но анестезиолог, например,...",0,отношение неплохой анестезиолог например прийт...,13,0.371662
8,"Ну, начнём.\nЗначит сегодня 26.08.2015 год.\nР...",0,начнм значить сегодня 26082015 год решить позв...,1,0.651210
9,Во время беременности мне необходимо было сдат...,1,время беременность необходимый сдать обязатель...,4,0.285010


In [92]:
top_docs_per_topic = df.groupby('topic_id').apply(
    lambda x: x.nlargest(min(5, len(x)), 'topic_confidence'), include_groups=False
).reset_index(drop=False)

In [94]:
top_docs_per_topic

,topic_id,level_1,reviews,sentiment,processed_reviews,topic_confidence
0,1,3214,Сегодня по направлению Вашего врача пришел сда...,0,сегодня направление ваш врач прийти сдавать ан...,0.811679
1,1,6844,"В понедельник, в терминале взяли талончик на с...",0,понедельник терминал взять талончик среда учас...,0.810256
2,1,4314,"На сайте этой аптеки уже давно ""висят"" объявле...",0,сайт аптека давно висеть объявление существова...,0.788203
3,1,2543,Сегодня получила незабываемые ощущения от рабо...,0,сегодня получить незабываемый ощущение работа ...,0.787075
4,1,6474,Хамское отношение в этой поликлиннике начинает...,0,хамский отношение поликлинника начинаться реги...,0.786495
5,3,4819,Добрый день! Хочу выразить огромную благодарно...,1,добрый день хотеть выразить огромный благодарн...,0.868307
6,3,8827,Хочу выразить огромную благодарность травматол...,1,хотеть выразить огромный благодарность травмат...,0.867327
7,3,9271,Благодарю от всей души за высокий профессионал...,1,благодарить весь душа высокий профессионализм ...,0.860578
8,3,6323,Выражаю сердечную признательность коллективу о...,1,выражать сердечный признательность коллектив о...,0.854459
9,3,2538,Большое спасибо врачам 17 гор. Больницы. Лежал...,1,большой спасибо врач 17 гор больница лежать тр...,0.843483


In [95]:
top_docs_per_topic.drop(columns=['level_1'])

,topic_id,reviews,sentiment,processed_reviews,topic_confidence
0,1,Сегодня по направлению Вашего врача пришел сда...,0,сегодня направление ваш врач прийти сдавать ан...,0.811679
1,1,"В понедельник, в терминале взяли талончик на с...",0,понедельник терминал взять талончик среда учас...,0.810256
2,1,"На сайте этой аптеки уже давно ""висят"" объявле...",0,сайт аптека давно висеть объявление существова...,0.788203
3,1,Сегодня получила незабываемые ощущения от рабо...,0,сегодня получить незабываемый ощущение работа ...,0.787075
4,1,Хамское отношение в этой поликлиннике начинает...,0,хамский отношение поликлинника начинаться реги...,0.786495
5,3,Добрый день! Хочу выразить огромную благодарно...,1,добрый день хотеть выразить огромный благодарн...,0.868307
6,3,Хочу выразить огромную благодарность травматол...,1,хотеть выразить огромный благодарность травмат...,0.867327
7,3,Благодарю от всей души за высокий профессионал...,1,благодарить весь душа высокий профессионализм ...,0.860578
8,3,Выражаю сердечную признательность коллективу о...,1,выражать сердечный признательность коллектив о...,0.854459
9,3,Большое спасибо врачам 17 гор. Больницы. Лежал...,1,большой спасибо врач 17 гор больница лежать тр...,0.843483


In [99]:
other_topics = [6, 12]
top_docs_per_topic['topic_id'] = top_docs_per_topic['topic_id'].apply(lambda x: 99 if x in other_topics else x)
df['topic_id'] = df['topic_id'].apply(lambda x: 99 if x in other_topics else x)

top_docs_per_topic

,topic_id,level_1,reviews,sentiment,processed_reviews,topic_confidence
0,1,3214,Сегодня по направлению Вашего врача пришел сда...,0,сегодня направление ваш врач прийти сдавать ан...,0.811679
1,1,6844,"В понедельник, в терминале взяли талончик на с...",0,понедельник терминал взять талончик среда учас...,0.810256
2,1,4314,"На сайте этой аптеки уже давно ""висят"" объявле...",0,сайт аптека давно висеть объявление существова...,0.788203
3,1,2543,Сегодня получила незабываемые ощущения от рабо...,0,сегодня получить незабываемый ощущение работа ...,0.787075
4,1,6474,Хамское отношение в этой поликлиннике начинает...,0,хамский отношение поликлинника начинаться реги...,0.786495
5,3,4819,Добрый день! Хочу выразить огромную благодарно...,1,добрый день хотеть выразить огромный благодарн...,0.868307
6,3,8827,Хочу выразить огромную благодарность травматол...,1,хотеть выразить огромный благодарность травмат...,0.867327
7,3,9271,Благодарю от всей души за высокий профессионал...,1,благодарить весь душа высокий профессионализм ...,0.860578
8,3,6323,Выражаю сердечную признательность коллективу о...,1,выражать сердечный признательность коллектив о...,0.854459
9,3,2538,Большое спасибо врачам 17 гор. Больницы. Лежал...,1,большой спасибо врач 17 гор больница лежать тр...,0.843483


### 3.2 Translation

In [100]:
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokeniser = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cpu"
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62518, 512, padding_idx=62517)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62518, 512, padding_idx=62517)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [101]:
def translate_text(texts, batch_size = 10):
    translations =[]
    for i in tqdm(range(0, len(texts), batch_size), desc = "translating"):
        batch = texts[i:i+batch_size]
        inputs = tokeniser(batch, return_tensors = "pt", padding = True, truncation = True).to(device)
        with torch.no_grad():
            translated = model.generate(**inputs)
        decoded = [tokeniser.decode(t, skip_special_tokens = True) for t in translated]
        translations.extend(decoded)
    return translations

In [102]:
reviews_list = top_docs_per_topic['reviews'].tolist()
translated_reviews = translate_text(reviews_list, batch_size = 10)

translating: 100%|████████████████████████████████| 6/6 [04:01<00:00, 40.27s/it]


In [103]:
top_docs_per_topic['translated_reviews'] = translated_reviews

In [104]:
top_docs_per_topic.to_csv("translated_reviews_by_topic.csv", index=False)

### 3.3 Summarisation

In [105]:
summariser = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

topic_summaries = []

for topic_id, group in tqdm(top_docs_per_topic.groupby('topic_id'), desc="Summarising topics"):
    
    reviews_text = " ".join(group['translated_reviews'].tolist())
    text_to_summarise = reviews_text
    
    summary = summariser(
        text_to_summarise,
        max_length=100,
        min_length=30,
        do_sample=False
    )[0]['summary_text']

    topic_summaries.append({
        "topic_id": topic_id,
        "summary": summary,
        "representative_reviews": reviews_text
    })


summaries_df = pd.DataFrame(topic_summaries)

print(summaries_df.head())

Device set to use mps:0
Summarising topics: 100%|███████████████████████| 10/10 [00:11<00:00,  1.20s/it]

   topic_id                                            summary  \
0         1   A patient was forced to pay 400 rubles to pay...   
1         3   I am ashamed of the professionalism of the do...   
2         4   I've cured a few teeth from Dr. Kurilov Evgen...   
3         7   6 r/d gave birth to a 17-year-old house in Lu...   
4        13   On November 22, 2015, a gynaecologist said th...   

                              representative_reviews  
0  Today, at your doctor's address, he came to do...  
1  Good afternoon, I wish to express my great gra...  
2  I've just had an unexpected positive experienc...  
3  Thanks to midwives, Tane and Vique, and all ot...  
4  Hello, I had a pregnancy for 11.5 weeks. On No...  


In [106]:
topic_stats = df.groupby('topic_id').agg(
    count = ('reviews', 'count'),
    positive = ('sentiment', lambda x: (x == 1).sum()),
    negative = ('sentiment', lambda x : (x == 0).sum())).reset_index()

In [107]:
topic_stats

,topic_id,count,positive,negative
0,1,1621,266,1355
1,3,2507,2455,52
2,4,2161,1832,329
3,7,208,194,14
4,13,2902,894,2008
5,15,119,39,80
6,17,28,16,12
7,18,188,121,67
8,19,256,23,233
9,99,10,2,8


In [108]:
final_df = summaries_df.merge(topic_stats, on='topic_id', how='left')

In [109]:
final_df

,topic_id,summary,representative_reviews,count,positive,negative
0,1,A patient was forced to pay 400 rubles to pay...,"Today, at your doctor's address, he came to do...",1621,266,1355
1,3,I am ashamed of the professionalism of the do...,"Good afternoon, I wish to express my great gra...",2507,2455,52
2,4,I've cured a few teeth from Dr. Kurilov Evgen...,I've just had an unexpected positive experienc...,2161,1832,329
3,7,6 r/d gave birth to a 17-year-old house in Lu...,"Thanks to midwives, Tane and Vique, and all ot...",208,194,14
4,13,"On November 22, 2015, a gynaecologist said th...","Hello, I had a pregnancy for 11.5 weeks. On No...",2902,894,2008
5,15,"Vladimir Pood. came with Caries in two teeth,...","Yeah, it's all beautiful and neat, the patient...",119,39,80
6,17,The T.C. Triumphal salon is not recommended f...,I bought contact lenses at the T.C. Triumphal ...,28,16,12
7,18,"After a sports injury, terrible back pains be...","After a sports injury, terrible back pains beg...",188,121,67
8,19,"A friend of ours died yesterday in ICU, and h...",There's no way to know the state of the patien...,256,23,233
9,99,"It's Child Protection Day again, and no one h...","It's Child Protection Day again, and no one ha...",10,2,8


In [ ]:
final_df.to_csv("../data/summaries.csv", index=False)